In [ ]:
# @title 連接google drive
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi
import os
os.chdir('/content/drive/My Drive')
os.getcwd()
os.chdir('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬')
os.getcwd()

Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found


'/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm

In [ ]:

df = pd.read_csv("filename.csv")
print(df)
print(df.info())
# df.describe(include="all")
print("object行:\n",df.select_dtypes(include="object").columns)
print("缺失值:\n",df.isnull().sum())
print("唯一值:\n",df[:].nunique())

            ID Delivery_person_ID Delivery_person_Age Delivery_person_Ratings  \
0      0x4607     INDORES13DEL02                   37                     4.9   
1      0xb379     BANGRES18DEL02                   34                     4.5   
2      0x5d6d     BANGRES19DEL01                   23                     4.4   
3      0x7a6a    COIMBRES13DEL02                   38                     4.7   
4      0x70a2     CHENRES12DEL01                   32                     4.6   
...        ...                ...                 ...                     ...   
45588  0x7c09      JAPRES04DEL01                   30                     4.8   
45589  0xd641      AGRRES16DEL01                   21                     4.6   
45590  0x4f8d     CHENRES08DEL03                   30                     4.9   
45591  0x5eee    COIMBRES11DEL01                   20                     4.7   
45592  0x5fb2   RANCHIRES09DEL02                   23                     4.9   

       Restaurant_latitude 

In [ ]:

def get_route_distance(lat1, lon1, lat2, lon2):
    # 构建请求URL
    url = f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false"

    # 发送请求
    response = requests.get(url)

    # 解析JSON数据
    data = response.json()

    # 检查是否有 'routes' 键
    if 'routes' in data and len(data['routes']) > 0:
        # 返回路径的距离，转换成公里
        return data['routes'][0]['distance'] / 1000
    else:
        # 没有找到有效路径，返回 None 或适当的错误信息
        return None

# 初始化 tqdm 对 pandas apply 方法的支持
tqdm.pandas(desc="计算距离")

# 使用 progress_apply 代替 apply 来使用进度条
df['Distance_km'] = df.progress_apply(lambda x: get_route_distance(x['Restaurant_latitude'], x['Restaurant_longitude'], x['Delivery_location_latitude'], x['Delivery_location_longitude']), axis=1)

print(df)


计算距离: 100%|██████████| 45593/45593 [5:55:59<00:00,  2.13it/s]

            ID Delivery_person_ID Delivery_person_Age Delivery_person_Ratings  \
0      0x4607     INDORES13DEL02                   37                     4.9   
1      0xb379     BANGRES18DEL02                   34                     4.5   
2      0x5d6d     BANGRES19DEL01                   23                     4.4   
3      0x7a6a    COIMBRES13DEL02                   38                     4.7   
4      0x70a2     CHENRES12DEL01                   32                     4.6   
...        ...                ...                 ...                     ...   
45588  0x7c09      JAPRES04DEL01                   30                     4.8   
45589  0xd641      AGRRES16DEL01                   21                     4.6   
45590  0x4f8d     CHENRES08DEL03                   30                     4.9   
45591  0x5eee    COIMBRES11DEL01                   20                     4.7   
45592  0x5fb2   RANCHIRES09DEL02                   23                     4.9   

       Restaurant_latitude 

In [ ]:
df
df.to_csv('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filename.csv', index=False)

In [ ]:

import requests
!pip install polyline
import polyline

def get_route(lat1, lon1, lat2, lon2):
    # Construct request URL
    url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=full"

    # Send request
    response = requests.get(url)

    # Parse JSON data
    data = response.json()

    # Check if the 'routes' key exists in the JSON response
    if 'routes' in data and len(data['routes']) > 0:
        # Decode Polyline
        route_polyline = data['routes'][0]['geometry']
        route = polyline.decode(route_polyline)

        # Transform coordinates format if necessary
        route = [[lat, lon] for lat, lon in route]

        return route
    else:
        # Return an empty list or None if no route is found, or handle the error as needed
        return None

from tqdm import tqdm
tqdm.pandas(desc="Calculating routes")

# Apply the function to the first 100 rows in the DataFrame to create a new 'Route' column
df['Route'] = df.loc[:1000].progress_apply(lambda x: get_route(x['Restaurant_latitude'], x['Restaurant_longitude'], x['Delivery_location_latitude'], x['Delivery_location_longitude']), axis=1)

df.to_csv('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filename1.csv', index=False)

Calculating routes: 100%|██████████| 1001/1001 [07:54<00:00,  2.11it/s]


In [ ]:
df.to_csv('/content/drive/My Drive/碳數據科技黑客松/總決賽/地圖模擬/filename1.csv', index=False)


In [ ]:

import folium
# !pip install polyline
import polyline


print(df['Route'])
# 獲取路徑
route = df['Route'][0]
print(df['Restaurant_latitude'][0]);print(df['Route'][0])
# 創建一個地圖對象，中心設定為起點
m = folium.Map(location=(df['Restaurant_latitude'][0],df['Restaurant_longitude'][0]), zoom_start=13)

# 繪製路線
folium.PolyLine(route, color='blue', weight=2.5, opacity=1).add_to(m)

# 顯示地圖
m

0        [[22.74505, 75.89247], [22.74507, 75.89248], [...
1        [[12.9131, 77.68297], [12.91196, 77.68272], [1...
2        [[12.91427, 77.67837], [12.91384, 77.67825], [...
3        [[11.00363, 76.97597], [11.00486, 76.97588], [...
4        [[12.97279, 80.25], [12.97335, 80.25014], [12....
                               ...                        
45588                                                  NaN
45589                                                  NaN
45590                                                  NaN
45591                                                  NaN
45592                                                  NaN
Name: Route, Length: 45593, dtype: object
22.745049
[[22.74505, 75.89247], [22.74507, 75.89248], [22.7452, 75.8925], [22.74526, 75.8925], [22.74567, 75.89262], [22.74557, 75.89301], [22.7454, 75.89364], [22.74594, 75.89381], [22.74673, 75.89407], [22.74711, 75.89419], [22.74724, 75.89423], [22.74845, 75.8946], [22.74885, 75.89471], [22.74895, 75.894

In [ ]:
from tqdm import tqdm

# 创建一个地图对象
m = folium.Map(location=[df.iloc[0]['Restaurant_latitude'], df.iloc[0]['Restaurant_longitude']], zoom_start=12)

# 使用 tqdm 包装 iterrows() 以显示进度条
for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc='Adding markers'):
    # 在餐厅位置添加红色标记
    folium.Marker(
        location=[row['Restaurant_latitude'], row['Restaurant_longitude']],
        icon=folium.Icon(color='red'),
        popup=f"Restaurant {idx}"
    ).add_to(m)

    # 在送货地点添加蓝色标记
    folium.Marker(
        location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
        icon=folium.Icon(color='blue'),
        popup=f"Delivery {idx}"
    ).add_to(m)

# 显示地图
m.save('map_with_progress.html')  # 保存地图为一个 HTML 文件


Adding markers:  38%|███▊      | 17165/45593 [00:04<00:08, 3470.58it/s]


KeyboardInterrupt: ignored

In [ ]:
m

In [ ]:
from tqdm import tqdm
import folium

# Assuming df is your DataFrame and it has a 'Route' column with polyline data
df=df.iloc[:1000]
# Create a map object centered on the first route's first coordinate
if df['Route'].notnull().any() and isinstance(df['Route'].dropna().iloc[0], list):
    map_center = df['Route'].dropna().iloc[0][0]
    m = folium.Map(location=map_center, zoom_start=13)
else:
    print("No valid route data available to create the map.")
    # You might want to exit the script or handle the error as needed

# Draw routes and add markers with a progress bar
for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing data'):
    # Check if route data exists and is a list
    if isinstance(row['Route'], list):
        # Draw the route
        folium.PolyLine(locations=row['Route'], color='blue', weight=5, opacity=0.5).add_to(m)
        # Add restaurant marker
        folium.Marker(
            location=[row['Restaurant_latitude'], row['Restaurant_longitude']],
            icon=folium.Icon(color='red'),
            popup=f"Restaurant {idx}"
        ).add_to(m)
        # Add delivery marker
        folium.Marker(
            location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
            icon=folium.Icon(color='blue'),
            popup=f"Delivery {idx}"
        ).add_to(m)

# Save the map to an HTML file
m.save('map_with_routes.html')
m

Output hidden; open in https://colab.research.google.com to view.